# 필수 프로그램 설치(Google Colab 에서 실행)

In [1]:
!pip3 install selenium selenium-wire  # 셀레니움 설치
!apt install chromium-browser # 크롬 브라우저 설치
!apt install chromium-chromedriver # 크롬 드라이버 설치
!cp /usr/lib/chromium-browser/chromedriver /usr/bin # 크롬 드라이버 복사

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-browser is already the newest version (101.0.4951.64-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (101.0.4951.64-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


# 프로그램 로드

## 배터리 패키지 로드
배터리: 프로그램을 설치하면 기본적으로 같이 설치되는 패키지들.
python을 설치한 후 pip를 하지 않고도 바로 import 할 수 있는 패키지를 지칭한다.

In [2]:
import time
import csv
from typing import List, NamedTuple, Tuple
import unicodedata
from random import randint

## 배터리 아닌 패키지 로드

In [3]:
import pandas as pd
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

import selenium
# from seleniumwire import webdriver
from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver import Firefox, FirefoxOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

## 브라우저 생성 함수
1. 어떤 브라우저를 실행시킬 것인가?
2. headless 모드(화면 없이 작동)를 활성화 할 것인가?

In [4]:
def get_browser():
    """브라우저 생성"""
    chrome_path = "/usr/bin/chromedriver"
    
    # Google Colab에서 실행시키기 위해서 옵션을 설정함
    # Windows에서 실행할 경우 알아서 설정하셔요
    options = webdriver.ChromeOptions()

    # 화면 표시 안함
    options.add_argument("headless")

    # 화면 크기
    options.add_argument('--windows-size=1920x1080')

    # ua 설정
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0')

    # 사생활 보호 모드로 실행
    # https://stackoverflow.com/questions/27630190/python-selenium-incognito-private-mode
    options.add_argument('--incognito')
    options.add_argument("--private")

    # 구글 콜랩을 위한 각종 에러 방지
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--ignore-ssl-errors')
    options.add_argument("--disable-popup-blocking")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    return webdriver.Chrome(chrome_path, options=options)

## 구글 드라이브와 연결한다

In [5]:
from google.colab import drive 

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


## 데이터를 저장하는 함수

In [6]:
def save_data(filename: str, data: list):
    df = pd.DataFrame([{"title": d["title"], "preview": "\n".join(d["preview"]).strip()} for d in data])
    df.to_json(f"/content/gdrive/My Drive/yes24.{filename}.json")
    df.to_json(f"/content/gdrive/My Drive/yes24.latest.json")

# 페이지 파싱하는 코드

In [7]:
# 전체 데이터를 담는 변수
data = []

# 이미 수집한 책인지 확인할수 있도록 제목만 저장하는 집합 변수
title_set = set()

## 베스트 목록에서 크롤링

In [8]:
def parse_page_from_best(browser):
    """베스트 목록에서 데이터를 추출한다."""
    # 리스트 추출
    elements = browser.find_elements(By.CSS_SELECTOR, "#category_layout > tbody:nth-child(1) > tr")

    # 목록이 없으면 끝까지 크롤링 완료했다고 여김
    if not elements:
        return False

    else:
        for elem in tqdm(elements[::2]):
            #---------- Selenium.START ----------
            # 제목 크롤링
            title_elem = elem.find_element(By.CSS_SELECTOR, "td.goodsTxtInfo > p > a")
            title = title_elem.get_attribute('innerHTML')
            #---------- Selenium.END ----------


            #---------- BeautifulSoup.START ----------
            soup = BeautifulSoup(elem.get_attribute('innerHTML'), "html.parser")

            # 미리보기 버튼이 있는지 확인함
            is_preview_availalbe = bool(soup.select_one('img[alt="이북 미리보기"]'))
            # 없으며 패스
            if not is_preview_availalbe:
                data.append({"title": title, "preview": []})
                continue
            #---------- BeautifulSoup.END ----------

            # 이미 수집한 제목은 넘김
            if title in title_set:
                continue
            else:
                title_set.add(title)

            #---------- Selenium.START ----------
            # 미리보기 열기
            preview_button = elem.find_element(By.CSS_SELECTOR, "td:nth-child(2) > div:nth-child(1) > a:nth-child(2)")
            js_code = preview_button.get_attribute('href')
            browser.execute_script(js_code.lstrip("javascript:"))

            # 미리보기 열릴 때까지 2초 대기
            time.sleep(2)

            # 팝업창으로 이동
            parent_window_id = browser.current_window_handle
            popup_window_id = browser.window_handles[1]
            browser.switch_to.window(popup_window_id)

            # 창 크기 조절
            browser.set_window_size(640, 320)
            #---------- Selenium.END ----------


            #---------- BeautifulSoup.START ----------
            preview_content = []
            try:
                # 미리보기 내용 가져옴
                preview_soup = BeautifulSoup(browser.page_source, "html.parser")
                for elem in preview_soup.select_one("div.viewer_window").select("li.chapter > p.txt"):
                    preview_content.append(elem.text.replace("\xa0", ""))
            except:
                print("미리보기 수집 실패:", title)
            #---------- BeautifulSoup.END ----------
            
            # 팝업창 닫기
            browser.close()
            
            # 원래 창으로 복귀
            browser.switch_to.window(parent_window_id)

            # 데이터를 리스트에 저장함
            data.append({"title": title, "preview": preview_content})
        return True

## 일반 소설 목록에서 크롤링

In [9]:
def parse_page_from_normal(browser):
    """일반 목록에서 데이터를 추출한다."""
    # 리스트 추출
    elements = browser.find_elements(By.CSS_SELECTOR, "#category_layout > ul.clearfix > li")

    # 목록이 없으면 끝까지 크롤링 완료했다고 여김
    if not elements:
        return False

    else:
        for elem in tqdm(elements):
            #---------- Selenium.START ----------
            # 제목 크롤링
            title_elem = elem.find_element(By.CSS_SELECTOR, "div.goods_info > div.goods_name > a")
            title = title_elem.get_attribute('innerHTML')
            #---------- Selenium.END ----------


            #---------- BeautifulSoup.START ----------
            soup = BeautifulSoup(elem.get_attribute('innerHTML'), "html.parser")

            # 미리보기 버튼이 있는지 확인함
            is_preview_availalbe = bool(soup.select_one('a.btnC.btn_preview > span > em'))
            # 없으며 패스
            if not is_preview_availalbe:
                data.append({"title": title, "preview": []})
                continue
            #---------- BeautifulSoup.END ----------

            # 이미 수집한 제목은 넘김
            if title in title_set:
                continue
            else:
                title_set.add(title)

            #---------- Selenium.START ----------
            # 미리보기 열기
            preview_button = elem.find_element(By.CSS_SELECTOR, "a.btnC.btn_preview")
            js_code = preview_button.get_attribute('href')
            browser.execute_script(js_code.lstrip("javascript:"))

            # 미리보기 열릴 때까지 2초 대기
            time.sleep(2)

            # 팝업창으로 이동
            parent_window_id = browser.current_window_handle
            popup_window_id = browser.window_handles[1]
            browser.switch_to.window(popup_window_id)

            # 창 크기 조절
            browser.set_window_size(640, 320)
            #---------- Selenium.END ----------


            #---------- BeautifulSoup.START ----------
            preview_content = []
            try:
                # 미리보기 내용 가져옴
                preview_soup = BeautifulSoup(browser.page_source, "html.parser")
                for div in soup.find_all("div", {'class':'author'}):
                    div.decompose()
                
                for elem in preview_soup.select_one("div.viewer_window").select("li.chapter > p"):
                    preview_content.append(elem.text.replace("\xa0", ""))
            except:
                print("미리보기 수집 실패:", title)
            #---------- BeautifulSoup.END ----------
            
            # 팝업창 닫기
            browser.close()
            
            # 원래 창으로 복귀
            browser.switch_to.window(parent_window_id)

            # 데이터를 리스트에 저장함
            data.append({"title": title, "preview": preview_content})
        return True

## 베스트 목록 크롤링 작동 코드

In [10]:
def crawling_at_best_list(min_page: int = 0, max_page: int = 1000):
    for i in range(min_page, max_page):
        url = "http://www.yes24.com/24/category/bestseller?CategoryNumber=017001045&sumgb=03&pagenumber={i + 1"
        print(i, "페이지 크롤링 中...")
        
        # 브라우저 열면서 크롤링 실시
        browser = get_browser()
        browser.get(url)
        
        # 과부하 방지를 위해 3초 휴식
        time.sleep(3)
        
        # 데이터 크롤링
        parse_page_from_best(browser)
        
        try:
            browser.close()
        except:
            pass
        try:
            print("미리보기 보유 비율:", (len([1 for d in data if d['preview'].strip()]) / len(data)) * 100, "%")
        except:
            pass
        
        save_data("best", data)

## 월별 베스트 목록 크롤링 작동 코드

In [11]:
def crawling_at_monthly_best_list(year: int = 2014, month: int = 5, min_page: int = 0, max_page: int = 1000):
    
    # 얼마나 남은지 모르므로 while
    while True:
        
        # 년월 계산
        if month == 12:
            year += 1
            month = 1
        else:
            month += 1
        
        # 미래인가?
        if year == 2022 and month == 7:
            # 종료
            break
        
        # url 생성
        url = f"http://www.yes24.com/24/category/bestseller?CategoryNumber=017001045&sumgb=09&year={year}&month={month}"

        
        for i in range(min_page, max_page):
            
            if i:
                url = url + f"&pagenumber={i + 1}"
            print(year, "년", month, "월", i, "페이지 크롤링 中...")
            
            # 브라우저 열면서 크롤링 실시
            browser = get_browser()
            browser.get(url)
            
            # 과부하 방지를 위해 3초 휴식
            time.sleep(3)
            
            try:
                # 데이터 크롤링
                if not parse_page_from_best(browser):
                    break
            except:
                pass
            finally:
                try:
                    browser.close()
                except:
                    pass

            save_data(f"monthly.{year}-{month}", data)

## 일반 목록 크롤링 작동 코드

In [12]:
def crawling_at_normal_list(category_id: str, name: str, min_page: int = 0, max_page = 1000):
    
    # 크롤링 할 페이지 지정
    for i in range(min_page, max_page):
        
        # 조건에 맞는 url 생성
        # FetchSize=40 40개를 로딩함
        # A0=2  성인용 제외
        # pagenumber=n 크롤링할 페이지 번호
        url = f"http://www.yes24.com/24/Category/Display/{category_id}?FetchSize=40&AO=2&pagenumber={i + 1}"
        print(name, i, "페이지 크롤링 中...")
        
        # 브라우저 열면서 크롤링 실시
        browser = get_browser()
        browser.get(url)
        
        # 과부하 방지를 위해 3초 휴식
        time.sleep(3)
        
        # 데이터 크롤링
        r = parse_page_from_normal(browser)
        
        try:
            browser.close()
        except:
            pass

        save_data("best", data)
        
        # 만약, 리스트가 비어 있었다면 크롤링 종료
        if not r:
            break

# 크롤링 대상 목록

In [13]:
target_list = [
               ("017001045006", "korean", 0, 1000),
               ("017001045007", "english", 0, 1000),
               ("017001045016", "detector", 0, 1000),
               ("017001045017", "sf", 0, 40),
               ("017001045022", "history", 0, 33),
               ("017001045018", "family", 0, 10),
               ("017001045019", "love", 0, 17),
               ("017001045020", "fairy_tail_for_older", 0, 10),
               ("017001045021", "from_movie", 0, 10),
               ("017001045024", "scenario", 0, 10),
]

# 크롤링

In [14]:
# yes24 베스트 목록 크롤링
crawling_at_best_list(0, 1)

# yes24 월별 베스트 목록 크롤링
crawling_at_monthly_best_list(2022, 5, 0, 1)

0 페이지 크롤링 中...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


  0%|          | 0/20 [00:00<?, ?it/s]

2022 년 6 월 0 페이지 크롤링 中...


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
for i in target_list:
    category_id = i[0]
    category_name = i[1]
    min_page = 0
    max_page = 1
    crawling_at_normal_list(category_id, category_name, min_page, max_page)

korean 0 페이지 크롤링 中...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


  0%|          | 0/40 [00:00<?, ?it/s]

english 0 페이지 크롤링 中...


  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

from_movie 0 페이지 크롤링 中...


  0%|          | 0/40 [00:00<?, ?it/s]

scenario 0 페이지 크롤링 中...


  0%|          | 0/40 [00:00<?, ?it/s]